In [ ]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

In [ ]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.visualization import plot_histogram
from qiskit.result import marginal_counts

# Teleportation

teleport = QuantumCircuit(3,3)

# qubit 0 is the state to be teleported from Alice
# qubits 1 and 2 are the shared entangled state
# Alice has qubit 1, Bob has qubit 2

# Add gates here to entangle qubits 1 and 2


# Add gates here to entangle qubit 0 with the others


# Measure qubits 0 and 1 and put the results on classical bits 0 and 1
teleport.measure([0,1],[0,1])

# Fix up the final state of qubit 2, depending on the measurement results
with teleport.if_test((1, 1)):
    teleport.x(2)
with teleport.if_test((0, 1)):
    teleport.z(2)

# Measure qubit 2 and put the result on classical bit 2
teleport.measure([2],[2])

# Check the circuit
teleport.draw("mpl")

In [ ]:
# Use a circuit to put qubit 0 into the state that we want to teleport,
# and compose it into the main circuit.

prep = QuantumCircuit(1) 
# Add a gate here to put qubit 0 into some state other than |0> 

teleport = teleport.compose(prep, [0], front=True)
teleport.draw("mpl")

In [ ]:
# Now simulate and plot the results.
# In general this won't tell us that we have exactly the right state,
# just that we get the right measurement statistics.
# We are interested in the result on classical bit 2, which is the most significant bit (leftmost).
# The simulator adds some noise, so we get some non-zero probabilities for parts of bit 2 that shouldn't be there.
# To see results without noise, add the parameter noise_info=False to GenericBackendV2

backend = GenericBackendV2(num_qubits=3, control_flow=True)
compiled = transpile(teleport, backend)
job_sim = backend.run(compiled, shots=1024)
result_sim = job_sim.result()
counts = result_sim.get_counts(compiled)
mcounts = marginal_counts(counts,indices=[2]) # We only want to see the results for bit 2
plot_histogram(mcounts)

In [ ]:
# Come back to this box if you have time after working through QC-Lab2-C

# Teleportation has an interesting feature which is not usually
# included in the description.

# If the qubit being teleported is entangled with other qubits, then after
# teleportation it is still entangled in the same way.

# Try this out!
# You will need to introduce another qubit and entangle it with qubit 0.
# After teleporting, work out how to use measurements to convince yourself that qubit 2 is entangled with the extra qubit.
# You can do this in a similar way to the last part of Lab1.